In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import torch
import numpy as np
from collections import Counter
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import math

## Additional RAM

In [ ]:
# But it is no more proposed get more RAM in a free way
[1]*10**10

## data

In [ ]:
books = "/content/drive/MyDrive/ProjetML/Base_BookCrossing/Base_completee/bothWebSites_InternetSearch_AllBooks_cleaned_w2v.csv"
ratings = "/content/drive/MyDrive/ProjetML/BX-Book-Ratings.csv"
ratings_bin = "/content/drive/MyDrive/ProjetML/BX-Book-Ratings-Binary.csv"

In [ ]:
#Cleaned data base with scrapped books
df_books = pd.read_csv(books, sep = "|")

In [ ]:
#ratings of Book Crossing
df_ratings = pd.read_csv(ratings, sep = ";", error_bad_lines=False, encoding="latin-1")

In [ ]:
print(len(df_ratings))
df_ratings.head()

1149780


,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [ ]:
df_books.head(1)

,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,average_rating,awards
0,0439023483,9.780439e+12,NaN,The Hunger Games : The First Book of the Hunge...,Suzanne Collins,2008-01-01,NaN,juvenile fict,NaN,NaN,en,http://books.google.com/books/content?id=sJdUA...,374.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_ratings[df_ratings.ISBN == "000648302X"]

,User-ID,ISBN,Book-Rating
45384,11676,000648302X,8
174443,37950,000648302X,9
373708,89624,000648302X,9
412327,98544,000648302X,0
417691,99328,000648302X,3
496961,119654,000648302X,8
499864,120610,000648302X,7
654880,158589,000648302X,8
659263,160016,000648302X,4
718973,174092,000648302X,0


# Tests 

### Tests merge

In [ ]:
DF_CatUsers = pd.DataFrame( {"UserID": [1,1,1,2,2,3,3,3,3], 
                             "ISBN": ["ISBN1", "ISBN2", "ISBN3", "ISBN4", "ISBN5", "ISBN2", "ISBN3", "ISBN5", "ISBN6"], 
                             "Ratings": [4,5,3,2,1,4,5,4,4]})
DF_CatUsers

In [ ]:
DF_CatBooks = pd.DataFrame( {"ISBN": ["ISBN1", "ISBN2", "ISBN3", "ISBN4", "ISBN5", "ISBN6", "ISBN7", "ISBN8"], 
                             "features": ["Blabla1", "Blabla2", "Blabla3", "Blabla4", "Blabla5", "Blabla6", "Blabla7", "Blabla8"]})
DF_CatBooks

In [ ]:
pd.merge(DF_CatUsers, DF_CatBooks, on = "ISBN")

### Test descriptions

In [ ]:
for d in df_books["book_description"][:3]:
    print(d)

# Bert on books descriptions (already trained)

### Loading

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.3MB 7.4MB/s 
     |████████████████████████████████| 3.3MB 24.7MB/s 
     |████████████████████████████████| 901kB 41.3MB/s 


In [ ]:
from transformers import BertTokenizer, BertModel

In [ ]:
#I will use an already trained BERT network
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

### Tokenizer and embedding

In [ ]:
#Use of pretrained model
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [ ]:
#With use of GPU I will save lots of time
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [ ]:
sent_list = df_bert.book_description[100].split(".")
sent_list = [sent for sent in sent_list if len(sent) > 3]
print(len(sent_list))
[sent for sent in sent_list if len(sent) > 3]

2


['This gripping tale unfolds in a peaceful Minnesota town, where crime is something that just doesn&#39;t happen',
 ' But when a young boy disappears, it marks the beginning of a unspeakable nightmare']

In [ ]:
"bonjour il n'y a pas de point".split(".")

["bonjour il n'y a pas de point"]

In [ ]:
df_bert = df_books.copy()

In [ ]:
lineOK = df_bert[(df_bert["book_description"].isna() == False) & (df_bert["book_language"] == "en")]
len(lineOK)

64229

In [ ]:
#Creation of an additional column in order to put the future book_description embedding
df_bert = df_bert.assign(description_bert = 0)

In [ ]:
#execution in about 16min

#from transformers import BertTokenizer
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

#model = BertModel.from_pretrained('bert-base-uncased'...
#forward(input_ids=None, attention_mask=None, ...

with torch.no_grad():
  for i_line in lineOK.index:
    
    #one book description, potentially composed of several sentences
    sent_list = df_bert.book_description[i_line].split(".")
    #I only keep the not empty results of the previous split
    sent_list = [sent for sent in sent_list if len(sent) > 3]

    #Output of sentences tokenization
    input_ids = []
    attention_masks = []

    #For each english sentence in the current book_description (not empty)
    ######################
    # Tokenization
    ######################
    for sent in sent_list:

      #print(len(sent.split()))
      #Tokenize all of the sentences and map the tokens to thier word IDs.
      encoded_dict = tokenizer.encode_plus(
                                  sent,                      # Sentence to encode.
                                  add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                                  max_length = 64,           # Pad & truncate all sentences.
                                  pad_to_max_length = True,
                                  return_attention_mask = True,   # Construct attn. masks.
                                  return_tensors = 'pt',     # Return pytorch tensors.
                        )
              
      # Add the encoded sentence to the list.    
      input_ids.append(encoded_dict['input_ids'])
              
      # And its attention mask (simply differentiates padding from non-padding).
      attention_masks.append(encoded_dict['attention_mask'])

    ######################
    # Embeddings
    ######################
    if (len(input_ids) > 0):
      # Convert the lists into tensors.
      input_ids_t = torch.cat(input_ids, dim = 0)
      attention_masks_t = torch.cat(attention_masks, dim = 0)

      #use Bert to embedds each tokenized sentences of the current book_description
      outputs = model(input_ids_t.to(device), attention_masks_t.to(device))
      hidden_states = outputs[2]

      # `hidden_states`       has shape [13 layers, len(input_ids), 64, 768] 
      # => hidden_states[-2]  has shape [len(input_ids), 64, 768] 
      token_vecs = hidden_states[-2]
      #print(token_vecs.shape)

      #mean elements by elements of each token (64 tokens in total) of each embedded sentences of the current book_description
      #=> sentence_embedding  has shape [len(input_ids), 768]: 
      sentence_embedding = torch.mean(token_vecs, dim=1)
      #print(sentence_embedding.shape)

      #I only want to keep one embedding of dim 768 for each book_description
      #(and not len(input_ids) x 768)
      sentence_embedding = torch.mean(sentence_embedding, dim=0)
      #print(sentence_embedding.size())


      ######################
      # Saving
      ######################
      #I get the values from GPU, and I put it in df_bert.description_bert column
      sentence_embedding = sentence_embedding.cpu().numpy()
      l = len(sentence_embedding)
      df_bert.description_bert[i_line] = [str(sentence_embedding[i]) for i in range(l)]


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats i

In [ ]:
# Print sentence 0, now as a list of IDs.
print('Original: ', df_bert.book_description[lineOK.index[0]])
print('Token IDs:', df_bert.description_bert[lineOK.index[0]])

Original:  With 160 million copies of the Twilight Saga sold worldwide, this addictive love story between a teenage girl and a vampire redefined romance for a generation. Here&#39;s the book that started it all.
Token IDs: ['-0.098250076', '-0.38007897', '0.8143984', '-0.069222644', '0.16267884', '-0.36887923', '0.09200936', '0.4111976', '0.32065162', '-0.0675951', '-0.17918313', '-0.24016543', '0.19495136', '0.4354491', '0.41287875', '0.4453775', '-0.24839687', '0.16070387', '0.14195046', '-0.16554643', '0.30131242', '0.28766793', '-0.12406763', '0.17628828', '0.21566428', '0.37729064', '-0.3119564', '0.008895561', '-0.51559365', '0.3141598', '0.25739694', '-0.24879043', '0.24140932', '0.0896724', '-0.38097662', '-0.012144949', '-0.15168047', '0.021698063', '-0.23981865', '0.23302785', '-0.5429584', '-0.34142113', '0.39717984', '-0.47054458', '-0.12300413', '-0.014472974', '0.20256266', '0.048919663', '-0.06945287', '-0.14903395', '-0.48877', '-0.08599328', '0.078301966', '-0.1906274'

In [ ]:
df_bert.to_csv("/content/drive/MyDrive/ProjetML/Books_description_bert.csv", sep = "|", index = False)

# Fast Text on books descriptions


### Sentences tokenized

In [ ]:
import re
import nltk

from nltk.stem import PorterStemmer
ps = PorterStemmer()

from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords_list = stopwords.words('english')

from nltk import WordPunctTokenizer
word_punctuation_tokenizer = nltk.WordPunctTokenizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def preprocess_text(document):
        # Remove all the special characters
        document = re.sub(r'\W', ' ', str(document))

        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

        # Remove single characters from the start
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

        # Substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)

        # Removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)

        # Converting to Lowercase
        document = document.lower()

        # Lemmatization
        tokens = document.split()
        tokens = [ps.stem(word) for word in tokens]
        tokens = [word for word in tokens if word not in stopwords_list]
        tokens = [word for word in tokens if len(word) > 3]

        preprocessed_text = ' '.join(tokens)

        return preprocessed_text

In [ ]:
#We only use the not empty book_description lines, and english books
lineOK = df_books[(df_books["book_description"].isna() == False) & (df_books["book_language"] == "en")]
len(lineOK)

64229

In [ ]:
print(len(lineOK))
df_books.book_description[lineOK.index[0]]

64229


'With 160 million copies of the Twilight Saga sold worldwide, this addictive love story between a teenage girl and a vampire redefined romance for a generation. Here&#39;s the book that started it all.'

In [ ]:
df_books.book_description[lineOK.index[1]]

'Bestselling author John Green returns with an eagerly anticipated and emotional novel about sickness and health, life and death.'

In [ ]:
#list of all sentences
final_corpus = []

for i_line in lineOK.index:
    
  #one book description, potentially composed of several sentences
  sent_list = df_books.book_description[i_line].split(".")

  #for each sub-sentences of the description: tokenization
  cleaned_sent = [preprocess_text(sent) for sent in sent_list if sent.strip() !='']

  #for each tokenized sub-sentences, a last cleaning
  word_tokenized_corpus = []
  for sent in cleaned_sent:
    word_tokenized_corpus.extend(word_punctuation_tokenizer.tokenize(sent))

  #only one list for all sub-sentences of the same book's description
  final_corpus.append(word_tokenized_corpus)


In [ ]:
cleaned_sent

['bestsel author john green return eagerli anticip emot novel sick health life death']

In [ ]:
final_corpus

[['million',
  'copi',
  'twilight',
  'saga',
  'sold',
  'worldwid',
  'addict',
  'love',
  'stori',
  'teenag',
  'girl',
  'vampir',
  'redefin',
  'romanc',
  'gener',
  'book',
  'start'],
 ['bestsel',
  'author',
  'john',
  'green',
  'return',
  'eagerli',
  'anticip',
  'emot',
  'novel',
  'sick',
  'health',
  'life',
  'death']]

In [ ]:
len(final_corpus)

64229

### Training

In [ ]:
from gensim.models.fasttext import FastText

In [ ]:
embedding_size = 60
window_size = 40
min_word = 1
down_sampling = 1e-2

In [ ]:
#It takes about 1H 10min for about 65 000 lines

# window_size :
#       is the size of the number of words occurring before and after 
#       the word based on which the word representations will be learned for the word.

#min_word :
#       which specifies the minimum frequency of a word in the corpus 
#       for which the word representations will be generated.

#sg :
#       A value of 1 specifies that we want to create skip-gram model.

ft_model = FastText(final_corpus,
                      size      = embedding_size,
                      window    = window_size,
                      min_count = min_word,
                      sample    = down_sampling,
                      sg        = 1,
                      iter      = 100)

In [ ]:
ft_model.save("/content/drive/MyDrive/ProjetML/FastText_desc.model")

### Save of embeddings

In [ ]:
ft_model = FastText.load("/content/drive/MyDrive/ProjetML/FastText_desc.model")

In [ ]:
df_fastFM = df_books.copy()

#Creation of an additional column in order to put the future book_description embedding
df_fastFM = df_fastFM.assign(description_fastFM = 0)

lineOK = df_books[(df_books["book_description"].isna() == False) & (df_books["book_language"] == "en")]
len(lineOK)

64229

In [ ]:
df_fastFM.head(1)

,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,average_rating,awards,description_fastFM
0,0439023483,9.780439e+12,NaN,The Hunger Games : The First Book of the Hunge...,Suzanne Collins,2008-01-01,NaN,juvenile fict,NaN,NaN,en,http://books.google.com/books/content?id=sJdUA...,374.0,NaN,NaN,NaN,NaN,NaN,0


In [ ]:
for i in range(len(lineOK.index)):

  #a tokenized sentences
  sent = final_corpus[i]
  #print("len(sent)", len(sent))

  if (len(sent) > 0):
    #find the embedding (vector of 60) for all words inside tokenized sentence
    words_emb = [ft_model.wv[w] for w in sent]
    #print(words_emb, len(words_emb))

    emb = [0.0]*60

    #Perform the sum of each words embeddings
    for word in words_emb:
      l = len(word)
      #sum of embedding of each words
      #print("\n", l, "\n", word)
      emb = [emb[k] + word[k] for k in range(l)]
      #print(emb)

    #print("emb", emb)
    #print("len(words_emb)", len(words_emb))
    emb = [emb[k] / len(words_emb) for k in range(l)]
    #print("emb", emb)  

    # Saving
    df_fastFM.description_fastFM[lineOK.index[i]] = [str(emb[i]) for i in range(l)]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
 [str(emb[i]) for i in range(l)]

['-0.015599380892056685',
 '0.34733651177241254',
 '-0.20130442784955868',
 '-0.254810130653473',
 '-0.0017622755840420723',
 '-0.17762010401258102',
 '-0.00482538715004921',
 '0.22535292173807436',
 '-0.01569059620109888',
 '0.04152290172462996',
 '-0.08608597660293946',
 '0.08708254095668402',
 '0.12789110868022993',
 '0.21732267533214047',
 '-0.2619647120053952',
 '-0.07633995605059542',
 '0.0036140303485668623',
 '-0.06340917027913608',
 '-0.027962678470290624',
 '-0.16541503068919367',
 '0.24956483608828142',
 '0.15758277624487305',
 '0.09344531438098504',
 '-0.012878868299034925',
 '0.03124704022103777',
 '-0.032272872156821765',
 '-0.1944929759662885',
 '-0.13054967041199023',
 '-0.3048797926077476',
 '-0.1617659333591851',
 '0.16737894513286078',
 '-0.16401336417318538',
 '0.29739678077972853',
 '-0.23934618956767595',
 '0.19287096021267083',
 '-0.26317760806817275',
 '-0.1648963175069254',
 '-0.10843969938846734',
 '0.010802279871243697',
 '0.33178424104475057',
 '0.0498662182

In [ ]:
# Print sentence 0, now as a list of IDs.
print('Original: ', df_fastFM.book_description[lineOK.index[0]])
print('Token IDs:', df_fastFM.description_fastFM[lineOK.index[0]])

Original:  With 160 million copies of the Twilight Saga sold worldwide, this addictive love story between a teenage girl and a vampire redefined romance for a generation. Here&#39;s the book that started it all.
Token IDs: ['0.046032028402859235', '0.15718470524777384', '-0.1395825319649542', '-0.1819172301055754', '0.17207935299066937', '-0.1365122384775211', '0.16689892704872525', '0.33930738577071357', '-0.06989275658612742', '0.31512450802085157', '-0.10388041380792856', '0.022007644121699473', '1.6696477199302e-05', '0.3797191175467828', '-0.273707770742476', '-0.20886098593473434', '0.11984747141490087', '-0.10271839490708183', '-0.09797142072142485', '-0.23660339820472634', '0.2803903391913456', '0.11936031935757081', '0.024216600488323498', '0.0775216789368321', '-0.014470523113713545', '-0.09681690356913297', '-0.09977425330811564', '0.1142483646159663', '-0.2905912635957493', '0.026502975954345483', '0.2798446424524574', '-0.09337197529042468', '0.21702254563570023', '-0.1523

In [ ]:
df_fastFM.to_csv("/content/drive/MyDrive/ProjetML/Books_description_fastText.csv", sep = "|", index = False)

# Fast Text on AuthorContext
- book_title 
- book_author 
- Category 
- Category_other 
- author_genres 
- author_genres_other

### Information preparation

In [ ]:
#We keep only few columns to create the context of training
df1 = df_books[df_books["book_language"] == "en"]
df1 = df1[["book_title", "book_author", "Category", "Category_other", "author_genres", "author_genres_other"]]
print(len(df_books))
print(len(df1))
df1[1000:1005]

105508
93808


,book_title,book_author,Category,Category_other,author_genres,author_genres_other
1010,Shadowland : The Immortals,Alyson Noël,young adult fict,NaN,young adult,NaN
1011,You Can Heal Your Life,Louise L. Hay,chemistri,psycholog,religion,spiritu
1013,I Am Legend,Richard Matheson,horror tal,NaN,science fict,fantasi
1014,Rules of Prey,John Sandford,fiction,NaN,mysteri,thriller
1015,The Yiddish Policemen's Union : A Novel,Michael Chabon,fiction,NaN,literatur,fiction


In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords_list = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#with data set of about 100 000 lines, it takes 1 min
lines_OK = df1.book_title[df1.book_title.isna() == False]

for i in lines_OK.index:
    #we get sentence inside book_title column, we remove final '.' and separate all words
    w = df1.book_title[i].strip('.').split()

    #we remove stopwords, words of length < 2 and '...'
    ww = [word.strip(',') for word in w if (word not in stopwords_list) and len(word) > 2 and (word != '...')]

    #We reconstruct a sentence with the remaining words, separated with ','
    df1.book_title[i] = ','.join(ww)

In [ ]:
print(len(df1))
df1.book_title.head()

93808


0    The,Hunger,Games,The,First,Book,Hunger,Games
1                                        Twilight
2                             The,Fault,Our,Stars
3                                   Angels,Demons
4                                       Divergent
Name: book_title, dtype: object

In [ ]:
#All those columns informations must be joined inside one column
#str(x) != "nan":     but we join only if column word is not empty
#x.split():           sentences in colum 'book_title' are cut in words
#','.join(x.split()): all those words are joined with (',') to be transformed after in list of list
def WordToJoin(w):
  ww = [x for x in w if str(x) != "nan"]
  return ','.join(ww)

df2 = df1.apply(lambda x: WordToJoin(x), axis = 1)
df2.head()

0    The,Hunger,Games,The,First,Book,Hunger,Games,S...
1    Twilight,Stephenie Meyer,young adult fict,scie...
2    The,Fault,Our,Stars,John Green,juvenile fict,l...
3     Angels,Demons,Dan Brown,fiction,mysteri,thriller
4    Divergent,Veronica Roth,juvenile fict,science ...
dtype: object

In [ ]:
#a specific data frame is created with all this train context
df_clean = pd.DataFrame({'clean': df2})
df_clean.head()

,clean
0,"The,Hunger,Games,The,First,Book,Hunger,Games,S..."
1,"Twilight,Stephenie Meyer,young adult fict,scie..."
2,"The,Fault,Our,Stars,John Green,juvenile fict,l..."
3,"Angels,Demons,Dan Brown,fiction,mysteri,thriller"
4,"Divergent,Veronica Roth,juvenile fict,science ..."


In [ ]:
#list of lists creation: 1 list for each book
sent = [' '.join(row.split(',')) for row in df_clean['clean']]
print(len(sent))
sent[0]

93808


'The Hunger Games The First Book Hunger Games Suzanne Collins juvenile fict'

### Training

In [ ]:
from gensim.models.fasttext import FastText

In [ ]:
embedding_size = 60
window_size = 40
min_word = 1
down_sampling = 1e-2

In [ ]:
#It takes about almost 4 Hours for about 105 000 lines

# window_size :
#       is the size of the number of words occurring before and after 
#       the word based on which the word representations will be learned for the word.

#min_word :
#       which specifies the minimum frequency of a word in the corpus 
#       for which the word representations will be generated.

#sg :
#       A value of 1 specifies that we want to create skip-gram model.

ft_model_auth = FastText(sent,
                      size      = embedding_size,
                      window    = window_size,
                      min_count = min_word,
                      sample    = down_sampling,
                      sg        = 1,
                      iter      = 100)

In [ ]:
ft_model_auth.save("/content/drive/MyDrive/ProjetML/FastText_auth.model")

### Save of embeddings

In [ ]:
ft_model_auth = FastText.load("/content/drive/MyDrive/ProjetML/FastText_auth.model")

In [ ]:
df_fastText_auth = df_books.copy()

#Creation of an additional column in order to put the future book_description embedding
df_fastText_auth = df_fastText_auth.assign(authorsContext_fastText = 0)

lineOK = df_fastText_auth[df_fastText_auth["book_language"] == "en"]
len(lineOK)

In [ ]:
print(len(df_fastText_auth))
print(len(sent))
df_fastText_auth.head(3)

105508
105508


,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,average_rating,awards,authorsContext_fastText
0,0439023483,9.780439e+12,NaN,The Hunger Games : The First Book of the Hunge...,Suzanne Collins,2008-01-01,NaN,juvenile fict,NaN,NaN,en,http://books.google.com/books/content?id=sJdUA...,374.0,NaN,NaN,NaN,NaN,NaN,0
1,0316015849,9.780316e+12,NaN,Twilight,Stephenie Meyer,2006-01-01,"Little, Brown Books for Young Readers",young adult fict,NaN,With 160 million copies of the Twilight Saga s...,en,NaN,544.0,science fict,fantasi,"['https://www.goodreads.com/book/show/41865', ...",3.61,Books I Loved Best Yearly,0
2,0525478817,9.780525e+12,NaN,The Fault in Our Stars,John Green,2012-01-01,Penguin,juvenile fict,NaN,Bestselling author John Green returns with an ...,en,http://books.google.com/books/content?id=Dc2LD...,318.0,literatur,fiction,[],4.19,West Australian Young Readers' Book Award,0


In [ ]:
df_fastText_auth.authorsContext_fastText[1]

0

In [ ]:
for i in range(len(lineOK.index)):

  #a tokenized sentences
  sent_auth = sent[i]
  #print("len(sent)", len(sent))

  if (len(sent_auth) > 0):
    #find the embedding (vector of 60) for all words inside tokenized sentence
    words_emb = [ft_model_auth.wv[w] for w in sent_auth]

    emb = [0.0]*60

    #Perform the sum of each words embeddings
    for word in words_emb:
      l = len(word)
      #sum of embedding of each words
      emb = [emb[k] + word[k] for k in range(l)]

    emb = [emb[k] / len(words_emb) for k in range(l)]

    # Saving
    df_fastText_auth.authorsContext_fastText[lineOK.index[i]] = [str(emb[i]) for i in range(l)]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
# Print sentence 0, now as a list of IDs.
print('Original: ', df_fastText_auth.book_title[lineOK.index[0]])
print('Token IDs:', df_fastText_auth.authorsContext_fastText[lineOK.index[0]])

In [ ]:
df_fastText_auth.to_csv("/content/drive/MyDrive/ProjetML/authorsContext_fastText.csv", sep = "|", index = False)

# Binary ratings

In [ ]:
#ratings of Book Crossing
df_ratings = pd.read_csv(ratings, sep = ";", error_bad_lines=False, encoding="latin-1")
print(len(df_ratings))
df_ratings.head(1)

1149780


,User-ID,ISBN,Book-Rating
0,276725,034545104X,0


In [ ]:
df_BinRatings = df_ratings.copy()
df_BinRatings["Book-Rating"].value_counts(sort = False )

0     716109
1       1770
2       2759
3       5996
4       8904
5      50974
6      36924
7      76457
8     103736
9      67541
10     78610
Name: Book-Rating, dtype: int64

In [ ]:
#Value 0 are put to 15 in order to be treated separately
#Then the two binary classes are created :
# 1, 2, 3 and 4 ratings become 0
# 7, 8, 9 and 10 ratings become 1

#Then a part of 0 previous rating will be used to balance the binary classes
#307000 lines of separated 15 values will be taken in random order to fill the binary classe 0
df_BinRatings["Book-Rating"] = df_BinRatings["Book-Rating"].map({0: 15, 1: 0, 2: 0, 3: 0, 4: 0, 7: 1, 8: 1, 9: 1, 10: 1})
df_BinRatings["Book-Rating"].value_counts(sort = False)

0.0      19429
15.0    716109
1.0     326344
Name: Book-Rating, dtype: int64

In [ ]:
#Values 15 (old 0 values) are selected...
df_BinRatings15 = df_BinRatings[df_BinRatings["Book-Rating"] == 15]
print(len(df_BinRatings15))
df_BinRatings15.head()

716109


,User-ID,ISBN,Book-Rating
0,276725,034545104X,15.0
2,276727,0446520802,15.0
5,276733,2080674722,15.0
10,276746,0425115801,15.0
11,276746,0449006522,15.0


In [ ]:
#... and put in random order
df_BinRatings15 = df_BinRatings15.sample(frac = 1, random_state=10).reset_index(drop = True)
print(len(df_BinRatings15))
df_BinRatings15.head()

716109


,User-ID,ISBN,Book-Rating
0,265202,0385471548,15.0
1,78974,8420400963,15.0
2,190807,0671042475,15.0
3,252695,0843950757,15.0
4,190708,044015393X,15.0


In [ ]:
#then only the first 307000 lines are kept in order to balance our future binary classes
df_BinRatings15 = df_BinRatings15[:307000]
print(len(df_BinRatings15))
df_BinRatings15.head()

307000


,User-ID,ISBN,Book-Rating
0,265202,0385471548,15.0
1,78974,8420400963,15.0
2,190807,0671042475,15.0
3,252695,0843950757,15.0
4,190708,044015393X,15.0


In [ ]:
#Then initial values 0 are put again
df_BinRatings15["Book-Rating"] = df_BinRatings15["Book-Rating"].map({15: 0})
print(len(df_BinRatings15))
df_BinRatings15.head()

307000


,User-ID,ISBN,Book-Rating
0,265202,0385471548,0
1,78974,8420400963,0
2,190807,0671042475,0
3,252695,0843950757,0
4,190708,044015393X,0


In [ ]:
#know we will concatenate new 0 classe...
df_BinRatings0 = df_BinRatings[df_BinRatings["Book-Rating"] == 0]
print(len(df_BinRatings0))
df_BinRatings0.head()

19429


,User-ID,ISBN,Book-Rating
3,276729,052165615X,0.0
49,276762,3453213025,0.0
55,276762,N3453124715,0.0
57,276768,9057868059,0.0
241,276853,8420457477,0.0


In [ ]:
#...with new 1 classe...
df_BinRatings1 = df_BinRatings[df_BinRatings["Book-Rating"] == 1]
print(len(df_BinRatings1))
df_BinRatings1.head()

326344


,User-ID,ISBN,Book-Rating
6,276736,3257224281,1.0
8,276744,038550120X,1.0
9,276745,342310538,1.0
16,276747,0060517794,1.0
19,276747,0671537458,1.0


In [ ]:
#...and with previous treated 0 values
df_ratings_new = pd.concat([df_BinRatings15, df_BinRatings0, df_BinRatings1], ignore_index = True, axis = 0)	
print(len(df_ratings_new))
df_ratings_new.head()

652773


,User-ID,ISBN,Book-Rating
0,265202,0385471548,0.0
1,78974,8420400963,0.0
2,190807,0671042475,0.0
3,252695,0843950757,0.0
4,190708,044015393X,0.0


In [ ]:
df_ratings_new["Book-Rating"].value_counts(sort = False )

0.0    326429
1.0    326344
Name: Book-Rating, dtype: int64

In [ ]:
df_ratings_new.to_csv("/content/drive/MyDrive/ProjetML/BX-Book-Ratings-Binary.csv", sep = "|", index = False)

# Ordinal regression

### Train / Test data creation

##### DF loadings

In [ ]:
#My_embedding = "bert"
#My_embedding = "fastText"
My_embedding = "fastText_auth"

In [ ]:
#We get the bert embedding of book_description 
if (My_embedding == "bert"):
  df_bert = pd.read_csv("/content/drive/MyDrive/ProjetML/Books_description_bert.csv", sep = "|")
  print(len(df_bert))

if (My_embedding == "fastText"):
  df_fastText = pd.read_csv("/content/drive/MyDrive/ProjetML/Books_description_fastText.csv", sep = "|")
  print(len(df_fastText))

if (My_embedding == "fastText_auth"):
  df_fastText_auth = pd.read_csv("/content/drive/MyDrive/ProjetML/authorsContext_fastText.csv", sep = "|")    
  print(len(df_fastText_auth))


105508


##### Merge

In [ ]:
#ratings of Book Crossing
#df_ratings = pd.read_csv(ratings, sep = ";", error_bad_lines=False, encoding="latin-1")
df_ratings = pd.read_csv(ratings_bin, sep = "|", error_bad_lines=False, encoding="latin-1")

print(len(df_ratings))
df_ratings.head(1)

652773


,User-ID,ISBN,Book-Rating
0,265202,0385471548,0.0


In [ ]:
#Join of ratings and books data base, according to ISBN which is a common column

if (My_embedding == "bert"):
  df_bert = pd.merge(df_ratings, df_bert, left_on = "ISBN", right_on = "isbn")  
  print(len(df_bert))
  
  #We only use the not empty book_description lines, and english books
  lineOK = df_bert[(df_bert["book_description"].isna() == False) & \
                   (df_bert["book_language"] == "en") & \
                   (df_bert["description_bert"] != '0')]
  print(len(lineOK))

if (My_embedding == "fastText"):
  df_fastText = pd.merge(df_ratings, df_fastText, left_on = "ISBN", right_on = "isbn")  
  print(len(df_fastText))

  #We only use the not empty book_description lines, and english books
  lineOK = df_fastText[(df_fastText["book_description"].isna() == False) & \
                       (df_fastText["book_language"] == "en") & \
                       (df_fastText["description_fastFM"] != '0')]
  print(len(lineOK))

if (My_embedding == "fastText_auth"):
  df_fastText_auth = pd.merge(df_ratings, df_fastText_auth, left_on = "ISBN", right_on = "isbn")  
  print(len(df_fastText_auth))

  #We only use the english books
  lineOK = df_fastText_auth[(df_fastText_auth["book_language"] == "en") & \
                            (df_fastText_auth["authorsContext_fastText"] != '0')]
  print(len(lineOK))

64148
47787


In [ ]:
My_embedding

'fastText_auth'

##### Columns selection

In [ ]:
if (My_embedding == "bert"):
  X_data = df_bert[["User-ID", "description_bert"]]

if (My_embedding == "fastText"):
  X_data = df_fastText[["User-ID", "description_fastFM"]]

if (My_embedding == "fastText_auth"):
  X_data = df_fastText_auth[["User-ID", "authorsContext_fastText"]]

X_data.head()

,User-ID,authorsContext_fastText
0,190708,"['-0.11191713949665427', '-0.08290330194202918..."
1,116599,"['-0.10984325339086354', '-0.08295069307088852..."
2,172030,"['-0.10984325339086354', '-0.08295069307088852..."
3,179733,"['-0.10984325339086354', '-0.08295069307088852..."
4,107784,"['-0.10984325339086354', '-0.08295069307088852..."


In [ ]:
X_data.iloc[1]

User-ID                                                               116599
authorsContext_fastText    ['-0.10984325339086354', '-0.08295069307088852...
Name: 1, dtype: object

##### Strings of embeddings transformed into float

In [ ]:
if (My_embedding == "bert"):
  s = 768
  MIN = min(df_bert["User-ID"])
  MAX = max(df_bert["User-ID"])
  print("min:", MIN)
  print("max:", MAX)

if (My_embedding == "fastText"):
  s = 60
  MIN = min(df_fastText["User-ID"])
  MAX = max(df_fastText["User-ID"])
  print("min:", MIN)
  print("max:", MAX)

if (My_embedding == "fastText_auth"):
  s = 60
  MIN = min(df_fastText_auth["User-ID"])
  MAX = max(df_fastText_auth["User-ID"])
  print("min:", MIN)
  print("max:", MAX)

min: 8
max: 278854


In [ ]:
#Transformation of a X_data line into a vector 
#beacause bert embedding has been saved as a list of strings
def vectBertfastText(l, s):

  #print(l, s)
  #for example bert embedding of s = 768 and User-ID
  b = np.zeros(s+1)

  #1st copy of User-ID
  b[0] = (l[0]-MIN)/(MAX-MIN)

  #Then handling of list of strings
  v = l[1].split("'")
  b[1:] = [float(v[i]) for i in range(1, len(v), 2)]
  
  return b

In [ ]:
res = vectBertfastText(X_data.iloc[1], s)
print("\n", len(res))
res[0:5]


 61


array([ 0.41811968, -0.10984325, -0.08295069, -0.05409687, -0.12666675])

In [ ]:
#treatment of all lines of X_data such that X_data.book_description is not empty
def matBertfastText(l, s, df):

  #for example bert embedding of s = 768 and User-ID
  m = np.zeros((len(l), (s+1)))
  
  ind = 0
  for i in l.index:
      #print(df.iloc[i])
      m[ind] = vectBertfastText(df.iloc[i], s)
      ind = ind + 1
  
  return m

In [ ]:
#It takes about 1min for about 75 000 lines
mat_X_data = matBertfastText(lineOK, s, X_data)
mat_X_data.shape

(47787, 61)

##### Labels lines selection

In [ ]:
if (My_embedding == "bert"):
  Y_data = df_bert["Book-Rating"]

if (My_embedding == "fastText"): 
  Y_data = df_fastText["Book-Rating"] 

if (My_embedding == "fastText_auth"): 
  Y_data = df_fastText_auth["Book-Rating"] 

Y_data.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: Book-Rating, dtype: float64

In [ ]:
#treatment of all corresponding lines of Y_data
#=> same lines as lines for which X_data.book_description is not empty
def vectLabels(l, df):
  m = np.zeros((len(l), 1), dtype = int)
  
  ind = 0
  for i in l.index:
      m[ind] = df.iloc[i]
      ind = ind + 1
  
  return m

In [ ]:
vect_Y_data = vectLabels(lineOK, Y_data)
vect_Y_data.shape

(47787, 1)

##### Train / Test split

In [ ]:
(X_train, X_test, y_train, y_test) = train_test_split(mat_X_data, vect_Y_data,          
                                           test_size=.2, stratify = vect_Y_data)

In [ ]:
print("len(lineOK)", len(lineOK), "\n")

print("X_train.shape", X_train.shape)
print("X_test.shape", X_test.shape, "\n")

print("y_train.shape", y_train.shape)
print("y_test.shape", y_test.shape)

len(lineOK) 47787 

X_train.shape (38229, 61)
X_test.shape (9558, 61) 

y_train.shape (38229, 1)
y_test.shape (9558, 1)


### Model training

##### Loading

In [ ]:
!pip install mord
from mord import LogisticAT, OrdinalRidge

In [ ]:
LogisticAT().get_params().keys()

dict_keys(['alpha', 'max_iter', 'verbose'])

In [ ]:
np.logspace(3, 5, 5)

array([  1000.        ,   3162.27766017,  10000.        ,  31622.77660168,
       100000.        ])

In [ ]:
dico_param = {
    'max_iter': np.logspace(3, 5, 5).astype(int),
    'alpha': np.logspace(0, 4, 5)
}

recherche_hyper = GridSearchCV(estimator = LogisticAT(), \
			param_grid = dico_param, scoring="accuracy", cv=5)

##### Unbalanced ratings

In [ ]:
X_train[0:5]

array([[ 9.84776543e-01, -1.17187218e-01, -8.48503003e-02,
        -4.10519843e-02, -1.27068600e-01, -3.49401187e-02,
         1.90372486e-01,  3.39874818e-04,  1.50763249e-01,
         8.39921434e-03, -2.28756532e-03,  1.65632713e-02,
         1.12846174e-01, -4.51982167e-02, -2.92917957e-02,
         1.08493861e-01, -6.48969447e-02, -1.49979662e-03,
         2.59884767e-02,  6.34772754e-03,  5.53870595e-02,
         1.23146707e-01,  4.63807447e-02, -9.93334434e-04,
         5.62731486e-03, -9.38865942e-02,  9.82192920e-02,
         1.47257158e-01,  6.34543662e-02, -4.35543111e-03,
         3.11393550e-02,  1.97914434e-02, -9.13288885e-03,
         3.68282168e-02, -9.41928322e-02, -9.18294066e-03,
         4.24706531e-02,  1.22308696e-01,  1.48180237e-02,
         5.74630152e-03, -9.12338948e-03,  2.29304854e-01,
         5.58225095e-02, -4.92421722e-02,  7.42736669e-02,
        -5.66707738e-02, -7.48944302e-03, -3.97715971e-02,
        -3.29815788e-02, -1.04057355e-02,  7.45169443e-0

In [ ]:
Counter(np.squeeze(y_train))

Counter({0: 41372,
         1: 98,
         2: 162,
         3: 366,
         4: 508,
         5: 2793,
         6: 2090,
         7: 4376,
         8: 6090,
         9: 3872,
         10: 5128})

In [ ]:
#It takes about 7 to 14 min 
LAT = LogisticAT(alpha=0., max_iter = 10000)
LAT.fit(X_train, np.squeeze(y_train))

ORD = OrdinalRidge(alpha=0., max_iter = 10000)
ORD.fit(X_train, np.squeeze(y_train))

In [ ]:
#It takes about 20min
recherche_hyper.fit(X_train, np.squeeze(y_train))

NameError: ignored

##### Binary ratings

In [ ]:
X_train[0:5]

array([[ 8.29604872e-01, -1.07108287e-01, -9.99123560e-02,
        -4.07762866e-02, -1.26517002e-01, -3.67573776e-02,
         1.77019291e-01, -3.22164219e-03,  1.53194207e-01,
         3.62040536e-03, -3.88715687e-03,  8.46036533e-03,
         1.18960979e-01, -4.79659588e-02, -1.93602501e-02,
         9.70487071e-02, -8.14320540e-02,  1.46676473e-03,
         2.85018421e-02,  1.64077787e-02,  4.70971592e-02,
         1.09111808e-01,  3.67489469e-02,  1.69651233e-02,
         1.00656186e-02, -9.55578680e-02,  9.33844132e-02,
         1.38678497e-01,  4.93410062e-02,  8.30083859e-03,
         3.59805417e-02,  5.16455439e-03, -1.25581923e-02,
         3.20714817e-02, -1.02175635e-01, -1.32452606e-02,
         5.40596773e-02,  1.26848748e-01, -5.21958772e-03,
         1.03050020e-02, -5.30410813e-03,  2.40483943e-01,
         4.32774418e-02, -5.64553643e-02,  8.31620683e-02,
        -5.49896930e-02, -8.58173250e-03, -4.43856761e-02,
        -1.64703260e-02, -1.42803882e-02,  8.17688618e-0

In [ ]:
Counter(np.squeeze(y_train))

Counter({0: 18763, 1: 19466})

In [ ]:
#It takes about 7 to 14 min 
LAT = LogisticAT(alpha=0., max_iter = 10000)
LAT.fit(X_train, np.squeeze(y_train))

ORD = OrdinalRidge(alpha=0., max_iter = 10000)
ORD.fit(X_train, np.squeeze(y_train))

OrdinalRidge(alpha=0.0, copy_X=True, fit_intercept=True, max_iter=10000,
             normalize=False, random_state=None, solver='auto', tol=0.001)

### Model validation

##### Unbalanced ratings

In [ ]:
def ConfMat(y_pred, y_true):
  #misclassified elements for each classes
  compResTab = np.zeros(11)

  for i in range (len(y_test)):
    #The correct classe is y_test[i], so we increment its counter of misclassified if y_lin_pred[i] is not the same
    compResTab[int(y_test[i])] = compResTab[int(y_test[i])] + (y_pred[i] != y_test[i])
  
  p = Counter(np.squeeze(y_test))
  print("Class \tMisclassified \t    Percentage")
  for i in range(11):
     print("{0}\t{1}\t\t\t{2}".format(i, compResTab[i], round(100*(compResTab[i]/p[i]), 2)))
  print("\nTOTAL\t{0}\t\t\t{1}".format(compResTab.sum(), round(100*compResTab.sum()/len(y_test),2)))

In [ ]:
X_test[0:5]

array([[ 2.72358219e-01, -1.12342381e-01, -8.84495277e-02,
        -5.43543801e-02, -1.32078478e-01, -4.48570742e-02,
         1.84085337e-01,  3.10736757e-03,  1.43471213e-01,
         4.80379415e-03, -4.57761314e-03,  1.20039026e-02,
         1.16341314e-01, -4.30509074e-02, -2.11059222e-02,
         1.09750352e-01, -7.40896770e-02,  3.30214084e-03,
         3.36847952e-02,  1.46281683e-02,  3.91453853e-02,
         1.10448458e-01,  3.47041068e-02,  4.98725409e-03,
        -1.17179103e-03, -9.75446040e-02,  9.62987116e-02,
         1.45353575e-01,  5.76217324e-02, -2.39527149e-03,
         2.89966920e-02,  2.33455593e-02, -1.34652704e-02,
         3.41409499e-02, -9.51513391e-02, -4.80232621e-03,
         5.10134121e-02,  1.24759698e-01,  1.16213474e-03,
         1.04870396e-02, -1.21704356e-02,  2.35315736e-01,
         4.04744382e-02, -5.65290036e-02,  7.67668866e-02,
        -5.73637488e-02, -2.67556962e-03, -4.71453100e-02,
        -2.52559907e-02, -1.51803410e-02,  7.82998547e-0

In [ ]:
Counter(np.squeeze(y_test))

Counter({0: 10343,
         1: 25,
         2: 40,
         3: 91,
         4: 127,
         5: 699,
         6: 522,
         7: 1094,
         8: 1523,
         9: 968,
         10: 1282})

In [ ]:
y_pred_lat = LAT.predict(X_test)
y_pred_ord = ORD.predict(X_test)
print(y_pred_lat.shape)
print(y_pred_ord.shape)

(16714,)
(16714,)


In [ ]:
y_pred = recherche_hyper.predict(X_test)
print(y_pred.shape)

NameError: ignored

In [ ]:
Counter(y_pred_lat)

Counter({0: 16440,
         1: 18,
         2: 29,
         3: 40,
         4: 54,
         5: 103,
         6: 19,
         7: 8,
         8: 1,
         9: 2})

In [ ]:
Counter(y_pred_ord)

Counter({0.0: 1, 2.0: 2466, 3.0: 12608, 4.0: 1617, 5.0: 19, 6.0: 1, 7.0: 2})

In [ ]:
Counter(y_pred)

NameError: ignored

In [ ]:
compRes = [i for i in range(len(y_test)) if y_pred_lat[i] != y_test[i]]
print(len(compRes))
print("Percentage of misclassified (LogisticAT): ", (len(compRes) / len(y_test))*100)

6491
Percentage of misclassified (LogisticAT):  38.8357065932751


In [ ]:
ConfMat(y_pred_lat, y_test)

Class 	Misclassified 	    Percentage
0	127.0			1.23
1	25.0			100.0
2	40.0			100.0
3	91.0			100.0
4	127.0			100.0
5	693.0			99.14
6	522.0			100.0
7	1094.0			100.0
8	1523.0			100.0
9	967.0			99.9
10	1282.0			100.0

TOTAL	6491.0			38.84


In [ ]:
compRes = [i for i in range(len(y_test)) if y_pred_ord[i] != y_test[i]]
print("Percentage of misclassified (OrdinalRidge): ", (len(compRes) / len(y_test))*100)

Percentage of misclassified (OrdinalRidge):  99.49742730644968


In [ ]:
ConfMat(y_pred_ord, y_test)

Class 	Misclassified 	    Percentage
0	10342.0			99.99
1	25.0			100.0
2	35.0			87.5
3	24.0			26.37
4	118.0			92.91
5	697.0			99.71
6	522.0			100.0
7	1094.0			100.0
8	1523.0			100.0
9	968.0			100.0
10	1282.0			100.0

TOTAL	16630.0			99.5


##### Binary ratings

In [157]:
def conf_pour(matr):
  conf_pour = matr
  conf_pour = conf_pour.astype('float64')
  conf_pour[0][0] = 100*(round(matr[0][0] / (matr[0][0] + matr[0][1]), 4))
  conf_pour[0][1] = 100*(round(matr[0][1] / (matr[0][0] + matr[0][1]), 4))
  conf_pour[1][0] = 100*(round(matr[1][0] / (matr[1][0] + matr[1][1]), 4))
  conf_pour[1][1] = 100*(round(matr[1][1] / (matr[1][0] + matr[1][1]), 4))
  print("Confusion matrix (LogisticAT) :", conf_pour, sep = "\n")

In [ ]:
X_test[0:5]

array([[ 4.87731579e-01, -2.75155395e-03,  2.35829036e-01,
        -1.71999358e-01, -2.24490861e-01, -5.65780554e-02,
        -9.39431704e-02,  4.64209423e-02,  2.62647065e-01,
        -2.57471128e-02,  1.25901811e-01, -1.81842809e-01,
         9.33393573e-02,  1.86756525e-02,  5.62890900e-01,
        -8.61057251e-02,  2.09481941e-01,  2.40493260e-02,
         2.09176090e-01,  6.10838905e-03, -1.43321593e-01,
         7.12078916e-02, -9.73540375e-02, -1.48026257e-01,
         1.51468646e-01,  7.62021814e-02, -2.82587484e-04,
        -9.37935216e-02, -2.23237971e-01, -3.60016279e-01,
        -1.21511299e-01,  3.18095073e-01, -1.66067153e-01,
         4.15998647e-01, -1.35019756e-01,  1.40138782e-01,
        -6.79547934e-02, -2.08313824e-01,  9.09398936e-02,
         5.26024797e-02,  3.42400144e-01,  4.24567328e-01,
         2.47933632e-03,  1.48328579e-01, -2.22357832e-01,
         2.21109226e-01, -1.81734525e-01,  5.42226484e-02,
        -3.87805125e-01,  6.48313361e-02, -1.63911883e-0

In [ ]:
Counter(np.squeeze(y_test))

Counter({0: 4691, 1: 4867})

In [ ]:
y_pred_lat = LAT.predict(X_test)
y_pred_ord = ORD.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix_lat = confusion_matrix( y_pred_lat, y_test )
print("Confusion matrix (LogisticAT) :", confusion_matrix_lat, sep = "\n")

Confusion matrix (LogisticAT) :
[[1810 1448]
 [1751 2193]]


In [158]:
confusion_matrix_lat = confusion_matrix( y_pred_lat, y_test )
print(confusion_matrix_lat)

conf_pour(confusion_matrix_lat)

[[2375 2013]
 [2316 2854]]
Confusion matrix (LogisticAT) :
[[54.12 45.88]
 [44.8  55.2 ]]


In [155]:
confusion_matrix_lat[0][1] / (confusion_matrix_lat[0][0] + confusion_matrix_lat[0][1])

0.4587511394712853

In [ ]:
round(conf_pour[0][0] / (conf_pour[0][0] + conf_pour[0][1]), 4)

0.3576

In [ ]:
print("Percentage of misclassified (LogisticAT): ", ((confusion_matrix_lat[0,1] + confusion_matrix_lat[1,0]) / len(y_test))*100)

Percentage of misclassified (LogisticAT):  45.29190207156309


In [159]:
confusion_matrix_ord = confusion_matrix( y_pred_ord, y_test )
print(confusion_matrix_ord)

conf_pour(confusion_matrix_ord)

[[2370 2003]
 [2321 2864]]
Confusion matrix (LogisticAT) :
[[54.2  45.8 ]
 [44.76 55.24]]


In [ ]:
print("Percentage of misclassified (OrdinalRidge): ", ((confusion_matrix_ord[0,1] + confusion_matrix_ord[1,0]) / len(y_test))*100)

Percentage of misclassified (OrdinalRidge):  45.23958987235823


# Classical regression

In [ ]:
def MyRound(x): 
  return math.ceil(x) if math.ceil(x) - x < x - math.floor(x) else math.floor(x)

#Counter(list(map(MyRound, y_lin_pred)))
#Counter(list(map(MyRound, y_lin_pred)))

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model_LinReg = LinearRegression()
model_LinReg.fit(X_train, np.squeeze(y_train))

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
y_lin_pred = model_LinReg.predict(X_test)

In [ ]:
y_lin_pred = list(map(MyRound, y_lin_pred))
Counter(y_lin_pred)

Counter({-2: 1, 2: 2466, 3: 12608, 4: 1617, 5: 19, 6: 1, 7: 2})

In [ ]:
Counter(np.squeeze(y_test))

Counter({0: 10343,
         1: 25,
         2: 40,
         3: 91,
         4: 127,
         5: 699,
         6: 522,
         7: 1094,
         8: 1523,
         9: 968,
         10: 1282})

In [ ]:
compRes = [i for i in range(len(y_test)) if y_lin_pred[i] != y_test[i]]
print("Percentage of misclassified (LinearRegression): ", (len(compRes) / len(y_test))*100)

Percentage of misclassified (LinearRegression):  99.50341031470623


In [ ]:
ConfMat(y_lin_pred, y_test)

Class 	Misclassified 	    Percentage
0	10343.0			100.0
1	25.0			100.0
2	35.0			87.5
3	24.0			26.37
4	118.0			92.91
5	697.0			99.71
6	522.0			100.0
7	1094.0			100.0
8	1523.0			100.0
9	968.0			100.0
10	1282.0			100.0

TOTAL	16631.0			99.5


# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
modele_rf = RandomForestClassifier()

##### Unbalanced ratings

In [ ]:
modele_rf.fit(X_train, np.squeeze(y_train))	

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
y_predict_rf = modele_rf.predict(X_test)
Counter(y_predict_rf)	

Counter({0: 11531,
         1: 18,
         2: 28,
         3: 69,
         4: 102,
         5: 561,
         6: 431,
         7: 833,
         8: 1302,
         9: 828,
         10: 1011})

In [ ]:
compRes = [i for i in range(len(y_test)) if y_predict_rf[i] != y_test[i]]
print("Percentage of misclassified (RandomForestClassifier): ", (len(compRes) / len(y_test))*100)

Percentage of misclassified (RandomForestClassifier):  53.39834868972119


In [ ]:
ConfMat(y_predict_rf, y_test) 

Class 	Misclassified 	    Percentage
0	3024.0			29.24
1	25.0			100.0
2	40.0			100.0
3	91.0			100.0
4	125.0			98.43
5	668.0			95.57
6	498.0			95.4
7	1033.0			94.42
8	1388.0			91.14
9	901.0			93.08
10	1132.0			88.3

TOTAL	8925.0			53.4


##### Binary ratings

In [163]:
modele_rf.fit(X_train, np.squeeze(y_train))	

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [164]:
y_predict_rf = modele_rf.predict(X_test)

In [165]:
confusion_matrix_rf = confusion_matrix( y_predict_rf, y_test )
print(confusion_matrix_rf)

conf_pour(confusion_matrix_rf)

[[2435 2133]
 [2256 2734]]
Confusion matrix (LogisticAT) :
[[53.31 46.69]
 [45.21 54.79]]


In [170]:
print("Percentage of misclassified (RandomForestClassifier): ", ((confusion_matrix_rf[0,1] + confusion_matrix_rf[1,0]) / len(y_test))*100)

Percentage of misclassified (RandomForestClassifier):  45.91964846202134


# Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
#no convergence with default solver ’lbfgs’ (even with max_iter = 10 000)
model_LogReg = LogisticRegression(max_iter = 10000, solver = "liblinear")

In [166]:
model_LogReg.fit(X_train, np.squeeze(y_train))	

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [167]:
y_predict_LogReg = model_LogReg.predict(X_test)

In [168]:
confusion_matrix_LogReg = confusion_matrix( y_predict_LogReg, y_test )
print(confusion_matrix_LogReg)

conf_pour(confusion_matrix_LogReg)

[[2065 1837]
 [2626 3030]]
Confusion matrix (LogisticAT) :
[[52.92 47.08]
 [46.43 53.57]]


In [169]:
print("Percentage of misclassified (LogisticRegression): ", ((confusion_matrix_LogReg[0,1] + confusion_matrix_LogReg[1,0]) / len(y_test))*100)

Percentage of misclassified (LogisticRegression):  46.693869010253195


# Average ratings by title

In [ ]:
val_title_mean = df_bert[["book_title", "Book-Rating"]].groupby("book_title").mean()	
Counter(list(map(MyRound, val_title_mean.values)))

Counter({0: 8165,
         1: 1039,
         2: 2355,
         3: 2813,
         4: 2288,
         5: 2047,
         6: 1146,
         7: 1344,
         8: 1504,
         9: 884,
         10: 1186})

# Test found on internet


In [ ]:
!pip install nose

     |████████████████████████████████| 163kB 22.4MB/s 


In [ ]:
import numpy as np
from scipy import stats, optimize, sparse
import mord
import functools
from nose.tools import assert_almost_equal, assert_greater_equal, assert_less

np.random.seed(0)
from sklearn import datasets, metrics, linear_model

In [ ]:
n_class = 5
n_samples = 100
n_dim = 10

In [ ]:
X = np.random.randn(n_samples, n_dim)
w = np.random.randn(n_dim)
y = X.dot(w)
bins = stats.mstats.mquantiles(y, np.linspace(0, 1, n_class + 1))
y = np.digitize(y, bins[:-1])
y -= y.min()

In [ ]:
def test_1():
    """
    Test two model in overfit mode
    """
    clf1 = mord.OrdinalRidge(alpha=0.)
    clf1.fit(X, y)

    clf2 = mord.LogisticAT(alpha=0.)
    clf2.fit(X, y)

    # the score is - absolute error, 0 is perfect
    # assert clf1.score(X, y) < clf2.score(X, y)

    clf3 = mord.LogisticSE(alpha=0.)
    clf3.fit(X, y)
    pred3 = clf3.predict(X)
    pred2 = clf2.predict(X)
    print("LogisticAT predictions \n", pred2)

    # check that it predicts better than the surrogate
    # for other loss
    assert np.abs(pred2 - y).mean() <= np.abs(pred3 - y).mean()
    # # the score is - absolute error, 0 is perfect
    # assert_almost_equal(clf.score(X, y), 0., places=2)
    #
    # clf = mord.LogisticIT(alpha=0.)
    # clf.fit(X, y)
    # # the score is classification error, 1 is perfect
    # assert_almost_equal(clf.score(X, y), 1., places=2)

    # test on sparse matrices
    X_sparse = sparse.csr_matrix(X)
    clf4 = mord.LogisticAT(alpha=0.)
    clf4.fit(X_sparse, y)
    pred4 = clf4.predict(X_sparse)
    assert metrics.mean_absolute_error(y, pred4) < 1.

In [ ]:
test_1()

LogisticAT predictions 
 [1 0 1 2 3 2 3 1 2 1 0 1 0 1 0 2 4 1 4 2 4 4 3 3 3 4 3 3 2 1 1 1 4 2 4 1 3
 4 1 0 4 4 3 0 3 2 2 0 3 0 0 4 0 2 3 1 3 0 4 2 0 4 3 0 1 1 0 1 2 4 2 2 1 0
 0 3 4 0 2 3 4 0 2 2 4 2 0 3 1 2 3 2 1 3 0 3 4 4 4 1]


In [ ]:
y

array([1, 0, 1, 2, 3, 2, 3, 1, 2, 1, 0, 1, 0, 1, 0, 2, 4, 1, 4, 2, 4, 4,
       3, 3, 3, 4, 3, 3, 2, 1, 1, 1, 4, 2, 4, 1, 3, 4, 1, 0, 4, 4, 3, 0,
       3, 2, 2, 0, 3, 0, 0, 4, 0, 2, 3, 1, 3, 0, 4, 2, 0, 4, 3, 0, 1, 1,
       0, 1, 2, 4, 2, 2, 1, 0, 0, 3, 4, 0, 2, 3, 4, 0, 2, 2, 4, 2, 0, 3,
       1, 2, 3, 2, 1, 3, 0, 3, 4, 4, 4, 1])

In [ ]:
X[0]

array([-1.00033035,  1.67398571,  0.16155927,  1.56340475, -0.79052302,
       -0.90730012,  0.22425222, -1.67868836,  0.21496559,  0.09721923])

In [ ]:
print(X.shape)
print(y.shape)

(100, 10)
(100,)


In [ ]:
Counter(y)

Counter({0: 20, 1: 20, 2: 20, 3: 20, 4: 20})